In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D, UpSampling2D, AveragePooling2D,ZeroPadding2D,Convolution2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
import tensorflow.keras.backend as K
from IPython.display import SVG, Image
import cv2
import utils

In [3]:
img_size=(100,100)
batch_size = 32
datagen_train=ImageDataGenerator(horizontal_flip=True)
train_generator=datagen_train.flow_from_directory("EyeTrack_Augmented/training/EyeTrack_Augmented",
target_size=img_size,
batch_size=batch_size,
class_mode='categorical',
color_mode ="grayscale",
shuffle=True)

datagen_validation=ImageDataGenerator(horizontal_flip=True)
validation_generator=datagen_validation.flow_from_directory("EyeTrack_Augmented/validation/EyeTrack_Augmented",
target_size=img_size,
batch_size=batch_size,
class_mode='categorical',
color_mode ="grayscale",
shuffle=True)

datagen_test=ImageDataGenerator(horizontal_flip=True)
test_generator=datagen_test.flow_from_directory("EyeTrack_Augmented/testing/EyeTrack_Augmented",
target_size=img_size,
batch_size=batch_size,
class_mode='categorical',
color_mode ="grayscale",
shuffle=True)

Found 2785 images belonging to 2 classes.
Found 347 images belonging to 2 classes.
Found 348 images belonging to 2 classes.


# MobileNetV2

In [4]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

In [5]:

mobilenetv2 = MobileNetV2(input_shape=[100,100]+ [1], weights=None, include_top=False)

In [6]:
# don't train existing weights
for layer in mobilenetv2.layers:
    layer.trainable = False

In [7]:
x = Flatten()(mobilenetv2.output)

In [9]:
prediction = Dense(2, activation='sigmoid')(x)

# create a model object
model = Model(inputs=mobilenetv2.input, outputs=prediction)

In [10]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100, 100, 1) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 50, 50, 32)   288         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 50, 50, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 50, 50, 32)   0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

In [11]:
def precision(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def recall(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall
optimum= Adam(learning_rate=0.000005)
model.compile(loss='binary_crossentropy', optimizer=optimum, metrics=['accuracy',recall,precision,tf.keras.metrics.AUC(curve="ROC"),tf.keras.metrics.FalsePositives(),tf.keras.metrics.TruePositives(),tf.keras.metrics.FalseNegatives(),tf.keras.metrics.TrueNegatives(),tf.keras.metrics.SensitivityAtSpecificity(0.5)])

In [12]:
%%time
batch_size = 32


ephocs=35
                    

steps_per_epoch=train_generator.n//train_generator.batch_size
steps_per_epoch
validation_steps=validation_generator.n//validation_generator.batch_size
validation_steps
mod = model.fit(x=train_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=ephocs,
                    validation_data=validation_generator,
                    validation_steps=validation_steps
                    )

Epoch 1/35
87/87 [==============================] - 14s 103ms/step - loss: 0.6931 - accuracy: 0.6023 - recall: 0.5995 - precision: 0.5995 - auc: 0.5000 - false_positives: 1083.0000 - true_positives: 1638.0000 - false_negatives: 1115.0000 - true_negatives: 1670.0000 - sensitivity_at_specificity: 0.0000e+00 - val_loss: 0.6931 - val_accuracy: 0.5406 - val_recall: 0.5406 - val_precision: 0.5406 - val_auc: 0.5000 - val_false_positives: 147.0000 - val_true_positives: 173.0000 - val_false_negatives: 147.0000 - val_true_negatives: 173.0000 - val_sensitivity_at_specificity: 0.0000e+00
Epoch 2/35
87/87 [==============================] - 8s 90ms/step - loss: 0.6931 - accuracy: 0.6001 - recall: 0.6045 - precision: 0.6045 - auc: 0.5000 - false_positives: 1101.0000 - true_positives: 1652.0000 - false_negatives: 1101.0000 - true_negatives: 1652.0000 - sensitivity_at_specificity: 0.0000e+00 - val_loss: 0.6931 - val_accuracy: 0.5625 - val_recall: 0.5625 - val_precision: 0.5625 - val_auc: 0.5000 - val_f

In [13]:
model.evaluate(test_generator,verbose=1)

11/11 [==============================] - 1s 110ms/step - loss: 0.6921 - accuracy: 0.5977 - recall: 0.5990 - precision: 0.5990 - auc: 0.5977 - false_positives: 140.0000 - true_positives: 208.0000 - false_negatives: 140.0000 - true_negatives: 208.0000 - sensitivity_at_specificity: 0.5977


[0.692110538482666,
 0.5977011322975159,
 0.5990259647369385,
 0.5990259647369385,
 0.5977011322975159,
 140.0,
 208.0,
 140.0,
 208.0,
 0.5977011322975159]

# EfficientNetB7

In [20]:
from tensorflow.keras.applications.efficientnet import EfficientNetB7
from tensorflow.keras.applications.efficientnet import preprocess_input


In [21]:

efficientnetb7 = EfficientNetB7(input_shape=[100,100]+ [1], weights=None, include_top=False)

In [22]:
# don't train existing weights
for layer in efficientnetb7.layers:
    layer.trainable = False

In [23]:
x = Flatten()(efficientnetb7.output)

In [24]:
prediction = Dense(2, activation='sigmoid')(x)

# create a model object
model = Model(inputs=efficientnetb7.input, outputs=prediction)

In [25]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 100, 100, 1) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 100, 100, 1)  0           input_3[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 100, 100, 1)  3           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 101, 101, 1)  0           normalization[0][0]              
____________________________________________________________________________________________

In [26]:
def precision(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def recall(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall
optimum= Adam(learning_rate=0.000005)
model.compile(loss='binary_crossentropy', optimizer=optimum, metrics=['accuracy',recall,precision,tf.keras.metrics.AUC(curve="ROC"),tf.keras.metrics.FalsePositives(),tf.keras.metrics.TruePositives(),tf.keras.metrics.FalseNegatives(),tf.keras.metrics.TrueNegatives(),tf.keras.metrics.SensitivityAtSpecificity(0.5)])

In [27]:
%%time
batch_size = 32


ephocs=35
                    

steps_per_epoch=train_generator.n//train_generator.batch_size
steps_per_epoch
validation_steps=validation_generator.n//validation_generator.batch_size
validation_steps
mod = model.fit(x=train_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=ephocs,
                    validation_data=validation_generator,
                    validation_steps=validation_steps
                    )

Epoch 1/35
87/87 [==============================] - 25s 140ms/step - loss: 0.6931 - accuracy: 0.5990 - recall: 0.6045 - precision: 0.6037 - auc_1: 0.5000 - false_positives_1: 1105.0000 - true_positives_1: 1652.0000 - false_negatives_1: 1101.0000 - true_negatives_1: 1648.0000 - sensitivity_at_specificity_1: 0.0000e+00 - val_loss: 0.6931 - val_accuracy: 0.5437 - val_recall: 0.5437 - val_precision: 0.5437 - val_auc_1: 0.5000 - val_false_positives_1: 146.0000 - val_true_positives_1: 174.0000 - val_false_negatives_1: 146.0000 - val_true_negatives_1: 174.0000 - val_sensitivity_at_specificity_1: 0.0000e+00
Epoch 2/35
87/87 [==============================] - 9s 99ms/step - loss: 0.6931 - accuracy: 0.6019 - recall: 0.5952 - precision: 0.5952 - auc_1: 0.5000 - false_positives_1: 1096.0000 - true_positives_1: 1657.0000 - false_negatives_1: 1096.0000 - true_negatives_1: 1657.0000 - sensitivity_at_specificity_1: 0.0000e+00 - val_loss: 0.6931 - val_accuracy: 0.5500 - val_recall: 0.5500 - val_precisi

In [28]:
model.evaluate(test_generator,verbose=1)

11/11 [==============================] - 1s 125ms/step - loss: 0.6921 - accuracy: 0.5977 - recall: 0.5974 - precision: 0.5974 - auc_1: 0.5977 - false_positives_1: 140.0000 - true_positives_1: 208.0000 - false_negatives_1: 140.0000 - true_negatives_1: 208.0000 - sensitivity_at_specificity_1: 0.5977


[0.6921136975288391,
 0.5977011322975159,
 0.5974026322364807,
 0.5974026322364807,
 0.5977011322975159,
 140.0,
 208.0,
 140.0,
 208.0,
 0.5977011322975159]

# Xception

In [29]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input


In [30]:
xception = Xception(input_shape=[100,100]+ [1], weights=None, include_top=False)

In [31]:
# don't train existing weights
for layer in xception.layers:
    layer.trainable = False

In [32]:
x = Flatten()(xception.output)

In [33]:
prediction = Dense(2, activation='sigmoid')(x)

# create a model object
model = Model(inputs=xception.input, outputs=prediction)

In [34]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 100, 100, 1) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 49, 49, 32)   288         input_4[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 49, 49, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 49, 49, 32)   0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

In [35]:
def precision(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def recall(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall
optimum= Adam(learning_rate=0.000005)
model.compile(loss='binary_crossentropy', optimizer=optimum, metrics=['accuracy',recall,precision,tf.keras.metrics.AUC(curve="ROC"),tf.keras.metrics.FalsePositives(),tf.keras.metrics.TruePositives(),tf.keras.metrics.FalseNegatives(),tf.keras.metrics.TrueNegatives(),tf.keras.metrics.SensitivityAtSpecificity(0.5)])

In [36]:
%%time
batch_size = 32


ephocs=35
                    

steps_per_epoch=train_generator.n//train_generator.batch_size
steps_per_epoch
validation_steps=validation_generator.n//validation_generator.batch_size
validation_steps
mod = model.fit(x=train_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=ephocs,
                    validation_data=validation_generator,
                    validation_steps=validation_steps
                    )

Epoch 1/35
87/87 [==============================] - 11s 101ms/step - loss: 0.6931 - accuracy: 0.6822 - recall: 0.8326 - precision: 0.6499 - auc_2: 0.5000 - false_positives_2: 1275.0000 - true_positives_2: 2287.0000 - false_negatives_2: 466.0000 - true_negatives_2: 1478.0000 - sensitivity_at_specificity_2: 0.0000e+00 - val_loss: 0.6931 - val_accuracy: 0.7563 - val_recall: 0.8281 - val_precision: 0.7039 - val_auc_2: 0.5000 - val_false_positives_2: 112.0000 - val_true_positives_2: 265.0000 - val_false_negatives_2: 55.0000 - val_true_negatives_2: 208.0000 - val_sensitivity_at_specificity_2: 0.0000e+00
Epoch 2/35
87/87 [==============================] - 8s 92ms/step - loss: 0.6930 - accuracy: 0.7940 - recall: 0.8542 - precision: 0.7416 - auc_2: 0.5000 - false_positives_2: 814.0000 - true_positives_2: 2347.0000 - false_negatives_2: 406.0000 - true_negatives_2: 1939.0000 - sensitivity_at_specificity_2: 0.0000e+00 - val_loss: 0.6930 - val_accuracy: 0.7781 - val_recall: 0.8125 - val_precision: 

In [37]:
model.evaluate(test_generator,verbose=1)

11/11 [==============================] - 1s 105ms/step - loss: 0.6898 - accuracy: 0.8218 - recall: 0.8218 - precision: 0.8150 - auc_2: 0.8119 - false_positives_2: 65.0000 - true_positives_2: 286.0000 - false_negatives_2: 62.0000 - true_negatives_2: 283.0000 - sensitivity_at_specificity_2: 0.4224


[0.6898247003555298,
 0.8218390941619873,
 0.82183438539505,
 0.8149549961090088,
 0.8118848204612732,
 65.0,
 286.0,
 62.0,
 283.0,
 0.42241379618644714]

# DenseNet201

In [38]:
from tensorflow.keras.applications.densenet import DenseNet201
from tensorflow.keras.applications.densenet import preprocess_input


In [39]:
densenet201 = DenseNet201(input_shape=[100,100]+ [1], weights=None, include_top=False)

In [40]:
# don't train existing weights
for layer in densenet201.layers:
    layer.trainable = False

In [41]:
x = Flatten()(densenet201.output)

In [42]:
prediction = Dense(2, activation='sigmoid')(x)

# create a model object
model = Model(inputs=densenet201.input, outputs=prediction)

In [43]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 100, 100, 1) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 106, 106, 1)  0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 50, 50, 64)   3136        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 50, 50, 64)   256         conv1/conv[0][0]                 
____________________________________________________________________________________________

In [44]:
def precision(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def recall(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall
optimum= Adam(learning_rate=0.000005)
model.compile(loss='binary_crossentropy', optimizer=optimum, metrics=['accuracy',recall,precision,tf.keras.metrics.AUC(curve="ROC"),tf.keras.metrics.FalsePositives(),tf.keras.metrics.TruePositives(),tf.keras.metrics.FalseNegatives(),tf.keras.metrics.TrueNegatives(),tf.keras.metrics.SensitivityAtSpecificity(0.5)])

In [45]:
%%time
batch_size = 32


ephocs=35
                    

steps_per_epoch=train_generator.n//train_generator.batch_size
steps_per_epoch
validation_steps=validation_generator.n//validation_generator.batch_size
validation_steps
mod = model.fit(x=train_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=ephocs,
                    validation_data=validation_generator,
                    validation_steps=validation_steps
                    )

Epoch 1/35
87/87 [==============================] - 19s 131ms/step - loss: 0.6791 - accuracy: 0.5721 - recall: 0.3193 - precision: 0.6425 - auc_3: 0.6195 - false_positives_3: 534.0000 - true_positives_3: 858.0000 - false_negatives_3: 1895.0000 - true_negatives_3: 2219.0000 - sensitivity_at_specificity_3: 0.6444 - val_loss: 0.6509 - val_accuracy: 0.5250 - val_recall: 0.5094 - val_precision: 0.5622 - val_auc_3: 0.6843 - val_false_positives_3: 126.0000 - val_true_positives_3: 163.0000 - val_false_negatives_3: 157.0000 - val_true_negatives_3: 194.0000 - val_sensitivity_at_specificity_3: 0.5406
Epoch 2/35
87/87 [==============================] - 9s 99ms/step - loss: 0.6426 - accuracy: 0.5074 - recall: 0.5244 - precision: 0.5345 - auc_3: 0.6650 - false_positives_3: 1272.0000 - true_positives_3: 1429.0000 - false_negatives_3: 1324.0000 - true_negatives_3: 1481.0000 - sensitivity_at_specificity_3: 0.4933 - val_loss: 0.6187 - val_accuracy: 0.5750 - val_recall: 0.5969 - val_precision: 0.5692 - v

In [46]:
model.evaluate(test_generator,verbose=1)

11/11 [==============================] - 2s 182ms/step - loss: 0.3078 - accuracy: 0.9195 - recall: 0.9221 - precision: 0.9196 - auc_3: 0.9756 - false_positives_3: 28.0000 - true_positives_3: 321.0000 - false_negatives_3: 27.0000 - true_negatives_3: 320.0000 - sensitivity_at_specificity_3: 0.9943


[0.3078140318393707,
 0.9195402264595032,
 0.9220779538154602,
 0.919581413269043,
 0.9756407737731934,
 28.0,
 321.0,
 27.0,
 320.0,
 0.9942528605461121]